## Construct a STANDARDIZED_LIST of input images and output labels

In [1]:
import cv2 # computer vision library
import helpers # helper functions

import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg # for loading in images

# Image data directories
IMAGE_DIR_TRAINING = "traffic_light_images/training/"
IMAGE_DIR_TEST = "traffic_light_images/test/"

# Using the load_dataset function in helpers.py
# Load training data
IMAGE_LIST = helpers.load_dataset(IMAGE_DIR_TRAINING)

# This function should take in an RGB image and return a new, standardized version
def standardize_input(image):
    
    ## TODO: Resize image and pre-process so that all "standard" images are the same size  
    standard_im = np.copy(image)
    
    standard_im = cv2.resize(image, (32,32))
    
    return standard_im

## TODO: One hot encode an image label
## Given a label - "red", "green", or "yellow" - return a one-hot encoded label
def one_hot_encode(label):
    
    ## TODO: Create a one-hot encoded label that works for all classes of traffic lights
    one_hot_encoded = [0 , 0 , 0] 
    
    if label is 'red':
        one_hot_encoded[0] = 1        
    elif label is 'yellow':
        one_hot_encoded[1] = 1
    else: #green
        one_hot_encoded[2] = 1  
    
    return one_hot_encoded

def standardize(image_list):
    
    # Empty image data array
    standard_list = []

    # Iterate through all the image-label pairs
    for item in image_list:
        image = item[0]
        label = item[1]

        # Standardize the image
        standardized_im = standardize_input(image)

        # One-hot encode the label
        one_hot_label = one_hot_encode(label)    

        # Append the image, and it's one hot encoded label to the full, processed list of image data 
        standard_list.append((standardized_im, one_hot_label))
        
    return standard_list

# Standardize all training images
STANDARDIZED_LIST = standardize(IMAGE_LIST)

## Histograms - contd

1. There will be 3 rows. One for each label. 
2. Each row, will have 4 cols. First for image, rest for HSV histograms

Earlier we just drew first image's HSV histogram for each label type. Here, we update for all images.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import display, Image
import time
%matplotlib notebook
rc('animation', html='html5')

# 3 rows, 4 cols
f, axArray = plt.subplots(3, 4, figsize=(7,3.5))
f.tight_layout() # http://tiny.cc/adjust-subplot-spaces
plt.close()  # to avoid an additional empty plot which we do not want to see

# --------------------- PREPROCESSING HELPER FUNCTIONS ----------------------

def crop(image_list):
    """
    crop 5 px on either side vertically
    """
    image_cropped_list = []
    for each_image_label_pair in image_list:

        image =  each_image_label_pair[0]        
        image_cropped = image[:, 5:-5, :]
        image_cropped_list.append((image_cropped, each_image_label_pair[1]))
        
    print(len(image_cropped_list))
    return image_cropped_list


# create separate list images for each channel
def create_separate_lists(image_list):
    """
    Returns 3 separate list of each label (so no label attachment)
    """

    r_list = []
    y_list = []
    g_list = []    
    for each_image_label_pair in image_list:

        image = each_image_label_pair[0]
        label = each_image_label_pair[1]  # one hot encoded

        if label[0] == 1: # red
            r_list.append(image)
        elif label[1] == 1: # yellow
            y_list.append(image)        
        else: # green
            g_list.append(image)        

    return (r_list, y_list, g_list)   

# ----------------- PRE PROCESSING SECTION ---------------------------
    
# CROP THE IMAGES
STANDARDIZED_CROPPED_LIST = crop(STANDARDIZED_LIST)

# create separate lists for each label
(r_list, y_list, g_list) = create_separate_lists(STANDARDIZED_CROPPED_LIST)

#print(len(r_list), len(y_list), len(g_list))

# get max length (list which has max no of images)
max_list = max([len(r_list), len(y_list), len(g_list)])


# ----------------- ANIMATION SECTION ------------------------------
# TO VIEW ALL THE PREPROCESSED IMAGES AT ONCE..

# calculate histogram
def hsv_histograms(rgb_image):
    # Convert to HSV
    hsv = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2HSV)

    # Create color channel histograms
    h_hist = np.histogram(hsv[:,:,0], bins=32, range=(0, 180))
    s_hist = np.histogram(hsv[:,:,1], bins=32, range=(0, 256))
    v_hist = np.histogram(hsv[:,:,2], bins=32, range=(0, 256))
    
    # Generating bin centers
    bin_edges = h_hist[1]
    bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2
    
    return bin_centers, h_hist, s_hist, v_hist

# initiate artists once
def initArtists():
    """
    To optimize performance, we use same artists, so they have to be initialized/created once
    like below and then re use in animation loop
    """
    axArtistsArray = [[plt.plot([],[]) for _ in range(4)] for _ in range(3)] 
    
    first_image_lists = [r_list[0], y_list[0], g_list[0]]    
    for i in range(3): # 3 rows        
        
        (bin_centers, h_hist, s_hist, v_hist) = hsv_histograms(first_image_lists[i])           
        
        axArtistsArray[i][0] = axArray[i][0].imshow(first_image_lists[i])                
        axArtistsArray[i][1] = axArray[i,1].bar(bin_centers,h_hist[0])
        axArtistsArray[i][2] = axArray[i,2].bar(bin_centers,s_hist[0])
        axArtistsArray[i][3] = axArray[i,3].bar(bin_centers,v_hist[0])
    
    return axArtistsArray

# animation function. This is called sequentially
def animate(i):

    # ensure no out of range in each lists
    r_index = i % len(r_list)
    y_index = i % len(y_list)
    g_index = i % len(g_list)
    
    first_image_lists = [r_list[r_index], y_list[y_index], g_list[g_index]]   
    """
    #activating below for loop, makes the code run for ever
    for i in range(3): # 3 rows        
        
        (bin_centers, h_hist, s_hist, v_hist) = hsv_histograms(first_image_lists[i])           
        
        axArtistsArray[i][0] = axArray[i][0].imshow(first_image_lists[i])                
        axArtistsArray[i][1] = axArray[i,1].bar(bin_centers,h_hist[0])
        axArtistsArray[i][2] = axArray[i,2].bar(bin_centers,s_hist[0])
        axArtistsArray[i][3] = axArray[i,3].bar(bin_centers,v_hist[0])    
        
    """
    axArtistsArray[0][0].set_data(r_list[r_index])
    axArtistsArray[1][0].set_data(y_list[y_index])
    axArtistsArray[2][0].set_data(g_list[g_index])
    
    return (axArtistsArray,)  


# INITIATE ARTISTS
axArtistsArray = initArtists()

#plt.subplots_adjust(hspace=None)

# call the animator. 
start_time = time.time()
anim = animation.FuncAnimation(f, animate, frames=np.arange(0,max_list), interval=500, blit=True)

# if output not cleared an empty output area created additionally...
from IPython.display import clear_output
clear_output()


anim

In [3]:
elapsed_time = time.time() - start_time
print('Elapsed time for above animation: {}'.format(elapsed_time))

Elapsed time for above animation: 122.75455474853516


In [4]:
print(plt.style.available)

['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', '_classic_test']
